# GASTO MEDIO - TRATAMIENTO

# Proposito:

En este notebook nos dedicaremos al tratamiento del gasto medio con el objetivo de establecer un estándar y limpiar los datos para quedarnos con lo que realmente nos interesa. 

In [1]:
%load_ext watermark
%watermark

Last updated: 2023-02-10T12:17:19.936727+00:00

Python implementation: CPython
Python version       : 3.8.5
IPython version      : 7.19.0

Compiler    : MSC v.1916 64 bit (AMD64)
OS          : Windows
Release     : 10
Machine     : AMD64
Processor   : Intel64 Family 6 Model 126 Stepping 5, GenuineIntel
CPU cores   : 8
Architecture: 64bit



In [2]:
import pandas as pd
import numpy as np
import os
import re

# PREPARACION PREVIA

Primero necesitamos copiar todos nuestros archivos en una carpeta donde podramos manipularlos para así conservar los archivos originales

In [3]:
from distutils.dir_util import copy_tree

# Método que limpia la carpeta de datos volatiles ya que es 
# una carpeta activa en todos los programas que hemos generado.
def limpiaCarpetaVolatil(rutaCopia):
    print("Limpiando Carpeta Volatil...")
    for f in os.listdir(rutaCopia):
        os.remove(os.path.join(rutaCopia, f))
        
#Metodo que copia recursivamente
def copia(dir_origen,dir_destino):
    print("Copiando...")
    copy_tree(dir_origen, dir_destino)
    print("Copiado")
    print(os.listdir(dir_destino))
  


In [4]:
rutaBase= "../PredictorGastoTuristico/DatosVirgenes/ISTAC/gasto/"
rutaCopia= "../PredictorGastoTuristico/DatosVolatiles/"

limpiaCarpetaVolatil(rutaCopia)
copia(rutaBase,rutaCopia)

Limpiando Carpeta Volatil...
Copiando...
Copiado
['gasto_medio_islas.csv']


In [5]:
gasto_medio_islas = pd.read_csv(rutaCopia+'gasto_medio_islas.csv',keep_default_na=False, na_values=[""])

In [6]:
gasto_medio_islas.head(10)

,fecha,pais,isla,total gasto por turista,gasto por turista alojamiento,gasto por turista transporte nacional / internacional,gasto por turista otros
0,2022 Tercer trimestre,TOTAL,Gran Canaria,"1.340,83","558,90","400,26","381,67"
1,2022 Segundo trimestre,TOTAL,Gran Canaria,"1.216,91","493,12","349,26","374,53"
2,2022 Primer trimestre,TOTAL,Gran Canaria,"1.440,08","645,89","370,33","423,86"
3,2021,TOTAL,Gran Canaria,"1.234,73","539,72","317,03","377,98"
4,2021 Cuarto trimestre,TOTAL,Gran Canaria,"1.274,38","575,45","340,81","358,12"
5,2021 Tercer trimestre,TOTAL,Gran Canaria,"1.143,95","485,78","295,00","363,17"
6,2021 Segundo trimestre,TOTAL,Gran Canaria,"1.197,36","477,30","284,06","436,00"
7,2021 Primer trimestre,TOTAL,Gran Canaria,"1.381,47","607,07","288,77","485,63"
8,2020,TOTAL,Gran Canaria,"1.253,77","564,23","324,24","365,30"
9,2020 Cuarto trimestre,TOTAL,Gran Canaria,"1.066,61","432,29","255,13","379,19"


In [7]:
gasto_medio_islas.shape

(2646, 7)

Visualizamos previamente los datos que tenemos: 

In [8]:
for columna in gasto_medio_islas:
    print(columna)
    print(gasto_medio_islas[columna].unique())
    print("\n")

fecha
['      2022 Tercer trimestre' '      2022 Segundo trimestre'
 '      2022 Primer trimestre' '2021' '      2021 Cuarto trimestre'
 '      2021 Tercer trimestre' '      2021 Segundo trimestre'
 '      2021 Primer trimestre' '2020' '      2020 Cuarto trimestre'
 '      2020 Tercer trimestre' '      2020 Segundo trimestre'
 '      2020 Primer trimestre' '2019' '      2019 Cuarto trimestre'
 '      2019 Tercer trimestre' '      2019 Segundo trimestre'
 '      2019 Primer trimestre' '2018' '      2018 Cuarto trimestre'
 '      2018 Tercer trimestre' '      2018 Segundo trimestre'
 '      2018 Primer trimestre' '2017' '      2017 Cuarto trimestre'
 '      2017 Tercer trimestre' '      2017 Segundo trimestre'
 '      2017 Primer trimestre' '2016' '      2016 Cuarto trimestre'
 '      2016 Tercer trimestre' '      2016 Segundo trimestre'
 '      2016 Primer trimestre' '2015' '      2015 Cuarto trimestre'
 '      2015 Tercer trimestre' '      2015 Segundo trimestre'
 '      2015 Primer tr

# UNIFICACIÓN Y ESTANDARIZACIÓN DE DATOS

## 1. FECHA

In [9]:
# A esta funcion se la pasa un dataframe, la columna y el elemento que queremos encontrar y remplazar en toda la columna
def replace_value(df, df_column_name, patron_to_find, patron_to_replace):
    try:
        my_list = []
        for item in df[df_column_name]: 
            #my_list.append(re.sub(r'(\d+)M(\d+)',r'\1-\2',item))
            my_list.append(re.sub(patron_to_find, patron_to_replace,item))
            
        print("Se ha cambiado el patron ({}) por el patron ({}) en la columna {}".format(
                patron_to_find, patron_to_replace, str(df[df_column_name].name)
            ))
        df[df_column_name] = pd.DataFrame(my_list)
        #print(df.head(5))
    except:
        print("Algo no funciona como deberia probablemente el error este en el patron")  

## 1.0 ESTANDARIZAMOS  (Problema dificíl de ver)
**IMPORTANTE LEER:**

Nos hemos percatado en posteriores códigos para hacer merge (Horas y horas pensado que el merge no funcionaba como debería) que en columna fecha había duplicados del cuartil Q1, y no tenia sentido porque ya habiamos eliminado las fechas anuales antes de transformar los cuartiles...

Vale pues resulta que **hay tres formatos**  en la columna fecha: 
  - '______2022 Tercer trimestre' (con espacios iniciales que no se ven)
  - '2010'
  - '_______2010' (con espacios iniciales)

Este ultimo no habia sido eliminado asique era el foco de problemas en uniones posteriores

Tiene unos espaciados que imposibilitan La Unión posterior de datos de tipo fecha así que vamos a tratarlo. esto nos enseña una vez más cómo el proceso de análisis de datos y tratamiento es un proceso en constante revisión y mejora que implica dar pasos hacia atrás debido a percatarnos de cosas que habíamos pasado previamente por alto.

In [10]:
gasto_medio_islas.fecha.unique()

array(['      2022 Tercer trimestre', '      2022 Segundo trimestre',
       '      2022 Primer trimestre', '2021',
       '      2021 Cuarto trimestre', '      2021 Tercer trimestre',
       '      2021 Segundo trimestre', '      2021 Primer trimestre',
       '2020', '      2020 Cuarto trimestre',
       '      2020 Tercer trimestre', '      2020 Segundo trimestre',
       '      2020 Primer trimestre', '2019',
       '      2019 Cuarto trimestre', '      2019 Tercer trimestre',
       '      2019 Segundo trimestre', '      2019 Primer trimestre',
       '2018', '      2018 Cuarto trimestre',
       '      2018 Tercer trimestre', '      2018 Segundo trimestre',
       '      2018 Primer trimestre', '2017',
       '      2017 Cuarto trimestre', '      2017 Tercer trimestre',
       '      2017 Segundo trimestre', '      2017 Primer trimestre',
       '2016', '      2016 Cuarto trimestre',
       '      2016 Tercer trimestre', '      2016 Segundo trimestre',
       '      2016 Primer t

In [11]:
dates=["2010","2011","2012","2013","2014","2015","2016","2017","2018","2019","2020","2021"]

for i in dates:
    pattern= r"\s+"+i
    p = r""+i
    replace_value(gasto_medio_islas, "fecha", pattern, p)

Se ha cambiado el patron (\s+2010) por el patron (2010) en la columna fecha
Se ha cambiado el patron (\s+2011) por el patron (2011) en la columna fecha
Se ha cambiado el patron (\s+2012) por el patron (2012) en la columna fecha
Se ha cambiado el patron (\s+2013) por el patron (2013) en la columna fecha
Se ha cambiado el patron (\s+2014) por el patron (2014) en la columna fecha
Se ha cambiado el patron (\s+2015) por el patron (2015) en la columna fecha
Se ha cambiado el patron (\s+2016) por el patron (2016) en la columna fecha
Se ha cambiado el patron (\s+2017) por el patron (2017) en la columna fecha
Se ha cambiado el patron (\s+2018) por el patron (2018) en la columna fecha
Se ha cambiado el patron (\s+2019) por el patron (2019) en la columna fecha
Se ha cambiado el patron (\s+2020) por el patron (2020) en la columna fecha
Se ha cambiado el patron (\s+2021) por el patron (2021) en la columna fecha


In [12]:
gasto_medio_islas.fecha.unique()

array(['      2022 Tercer trimestre', '      2022 Segundo trimestre',
       '      2022 Primer trimestre', '2021', '2021 Cuarto trimestre',
       '2021 Tercer trimestre', '2021 Segundo trimestre',
       '2021 Primer trimestre', '2020', '2020 Cuarto trimestre',
       '2020 Tercer trimestre', '2020 Segundo trimestre',
       '2020 Primer trimestre', '2019', '2019 Cuarto trimestre',
       '2019 Tercer trimestre', '2019 Segundo trimestre',
       '2019 Primer trimestre', '2018', '2018 Cuarto trimestre',
       '2018 Tercer trimestre', '2018 Segundo trimestre',
       '2018 Primer trimestre', '2017', '2017 Cuarto trimestre',
       '2017 Tercer trimestre', '2017 Segundo trimestre',
       '2017 Primer trimestre', '2016', '2016 Cuarto trimestre',
       '2016 Tercer trimestre', '2016 Segundo trimestre',
       '2016 Primer trimestre', '2015', '2015 Cuarto trimestre',
       '2015 Tercer trimestre', '2015 Segundo trimestre',
       '2015 Primer trimestre', '2014', '2014 Cuarto trimestre'

Se han quitado los espacios y vemos como no se ha modificado nada mas...

### 1.1 Transformamos la fecha a un formato usable Year-month

Vamos a estandarizar la fecha usando los valores trimestrales, pero en el formato estándar de cuartiles. Ya que vemos que tiene el formato "2020 Primer Trimestre"

Tratamos turistas_motivos['fecha'] cambiando las fechas por nuestro patron donde ponemos el primer mes que representa el inicio del periodo en cuartiles, para la posterior transformación.

In [13]:
replace_value(gasto_medio_islas, "fecha", r'(\d+) Primer trimestre', r'\1-01')
replace_value(gasto_medio_islas, "fecha", r'(\d+) Segundo trimestre', r'\1-04')
replace_value(gasto_medio_islas, "fecha", r'(\d+) Tercer trimestre', r'\1-07')
replace_value(gasto_medio_islas, "fecha", r'(\d+) Cuarto trimestre', r'\1-10')

gasto_medio_islas.head(10)

Se ha cambiado el patron ((\d+) Primer trimestre) por el patron (\1-01) en la columna fecha
Se ha cambiado el patron ((\d+) Segundo trimestre) por el patron (\1-04) en la columna fecha
Se ha cambiado el patron ((\d+) Tercer trimestre) por el patron (\1-07) en la columna fecha
Se ha cambiado el patron ((\d+) Cuarto trimestre) por el patron (\1-10) en la columna fecha


,fecha,pais,isla,total gasto por turista,gasto por turista alojamiento,gasto por turista transporte nacional / internacional,gasto por turista otros
0,2022-07,TOTAL,Gran Canaria,"1.340,83","558,90","400,26","381,67"
1,2022-04,TOTAL,Gran Canaria,"1.216,91","493,12","349,26","374,53"
2,2022-01,TOTAL,Gran Canaria,"1.440,08","645,89","370,33","423,86"
3,2021,TOTAL,Gran Canaria,"1.234,73","539,72","317,03","377,98"
4,2021-10,TOTAL,Gran Canaria,"1.274,38","575,45","340,81","358,12"
5,2021-07,TOTAL,Gran Canaria,"1.143,95","485,78","295,00","363,17"
6,2021-04,TOTAL,Gran Canaria,"1.197,36","477,30","284,06","436,00"
7,2021-01,TOTAL,Gran Canaria,"1.381,47","607,07","288,77","485,63"
8,2020,TOTAL,Gran Canaria,"1.253,77","564,23","324,24","365,30"
9,2020-10,TOTAL,Gran Canaria,"1.066,61","432,29","255,13","379,19"


### 1.2 Eliminamos fechas anuales

Eliminamos las fechas anuales ya que nos vamos a centrar en las fechas por trimestres. Para ello, primero convertimos la columna fecha de object a string.

In [14]:
gasto_medio_islas.fecha.astype("string")

0             2022-07
1             2022-04
2             2022-01
3                2021
4             2021-10
            ...      
2641             2010
2642          2010-10
2643          2010-07
2644          2010-04
2645          2010-01
Name: fecha, Length: 2646, dtype: string

Ahora buscamos las fechas anuales y las eliminamos:

In [15]:
#CODIGO INEFICIENTE QUE USAMOS EN OTRO JUPYTER NOTEBOOK PERO CON MUCHOS DATOS TARDA LA VIDA
# Y CLARO ME DI CUENTA QUE HACER ESTE CODIGO ES LO MISMO QUE USAR EL ISIN DE PYTHON... :)


##dates=["2010","2011","2012","2013","2014","2015","2016","2017","2018","2019","2020","2021","2022"]
##
##for row in gasto_medio_islas.fecha:
##    for year in dates:
##        if row == year:
##            gasto_medio_islas.drop(gasto_medio_islas[gasto_medio_islas['fecha'] == year].index,  axis=0, inplace=True)
##            break
##gasto_medio_islas.head(7)

#### SOLUCION A LO ANTERIOR...
#### ¿COMO?
**Filtrando con ~isin** que elimina filas que estan en la lista

In [16]:
dates=["2010","2011","2012","2013","2014","2015","2016","2017","2018","2019","2020","2021","2022"]

last_shape = gasto_medio_islas.shape

gasto_medio_islas = gasto_medio_islas[~gasto_medio_islas.fecha.isin(dates)] 
gasto_medio_islas.reset_index(drop=True, inplace=True)

actual_shape = gasto_medio_islas.shape
    
print("SE HAN ELIMINADO FECHAS ANUALES \n El archivo tenia: {} y ahora tiene {}".format(last_shape,actual_shape))

SE HAN ELIMINADO FECHAS ANUALES 
 El archivo tenia: (2646, 7) y ahora tiene (2142, 7)


In [17]:
gasto_medio_islas.head(2)

,fecha,pais,isla,total gasto por turista,gasto por turista alojamiento,gasto por turista transporte nacional / internacional,gasto por turista otros
0,2022-07,TOTAL,Gran Canaria,"1.340,83","558,90","400,26","381,67"
1,2022-04,TOTAL,Gran Canaria,"1.216,91","493,12","349,26","374,53"


### 1.3 Transformamos a cuartiles

Convertimos de datatime a a quarter

In [18]:
gasto_medio_islas['fecha'] = pd.PeriodIndex(gasto_medio_islas.fecha, freq="Q")

In [19]:
gasto_medio_islas.head(7)

,fecha,pais,isla,total gasto por turista,gasto por turista alojamiento,gasto por turista transporte nacional / internacional,gasto por turista otros
0,2022Q3,TOTAL,Gran Canaria,"1.340,83","558,90","400,26","381,67"
1,2022Q2,TOTAL,Gran Canaria,"1.216,91","493,12","349,26","374,53"
2,2022Q1,TOTAL,Gran Canaria,"1.440,08","645,89","370,33","423,86"
3,2021Q4,TOTAL,Gran Canaria,"1.274,38","575,45","340,81","358,12"
4,2021Q3,TOTAL,Gran Canaria,"1.143,95","485,78","295,00","363,17"
5,2021Q2,TOTAL,Gran Canaria,"1.197,36","477,30","284,06","436,00"
6,2021Q1,TOTAL,Gran Canaria,"1.381,47","607,07","288,77","485,63"


In [20]:
gasto_medio_islas.shape

(2142, 7)

### 1.4 Eliminamos cuartiles de 2020 y 2021 al ser anomalos por el covi-19

In [21]:
gasto_medio_islas.head()    

,fecha,pais,isla,total gasto por turista,gasto por turista alojamiento,gasto por turista transporte nacional / internacional,gasto por turista otros
0,2022Q3,TOTAL,Gran Canaria,"1.340,83","558,90","400,26","381,67"
1,2022Q2,TOTAL,Gran Canaria,"1.216,91","493,12","349,26","374,53"
2,2022Q1,TOTAL,Gran Canaria,"1.440,08","645,89","370,33","423,86"
3,2021Q4,TOTAL,Gran Canaria,"1.274,38","575,45","340,81","358,12"
4,2021Q3,TOTAL,Gran Canaria,"1.143,95","485,78","295,00","363,17"


In [22]:
gasto_medio_islas.shape

(2142, 7)

Vemos el rango que usaremos como filtro

In [23]:
pd.period_range(start=pd.Period('2010Q1', freq='Q'),end=pd.Period('2019Q4', freq='Q'))

PeriodIndex(['2010Q1', '2010Q2', '2010Q3', '2010Q4', '2011Q1', '2011Q2',
             '2011Q3', '2011Q4', '2012Q1', '2012Q2', '2012Q3', '2012Q4',
             '2013Q1', '2013Q2', '2013Q3', '2013Q4', '2014Q1', '2014Q2',
             '2014Q3', '2014Q4', '2015Q1', '2015Q2', '2015Q3', '2015Q4',
             '2016Q1', '2016Q2', '2016Q3', '2016Q4', '2017Q1', '2017Q2',
             '2017Q3', '2017Q4', '2018Q1', '2018Q2', '2018Q3', '2018Q4',
             '2019Q1', '2019Q2', '2019Q3', '2019Q4'],
            dtype='period[Q-DEC]', freq='Q-DEC')

In [24]:
#Aqui cojo el año y filtro eliminadno 2020 y 2021
gasto_medio_islas = gasto_medio_islas[gasto_medio_islas.fecha.isin(pd.period_range(start = pd.Period('2010Q1', freq='Q'), end = pd.Period('2019Q4', freq='Q')))]
gasto_medio_islas.reset_index(drop=True, inplace=True)   #podemos resetear el index del DataFrame
gasto_medio_islas.head()

,fecha,pais,isla,total gasto por turista,gasto por turista alojamiento,gasto por turista transporte nacional / internacional,gasto por turista otros
0,2019Q4,TOTAL,Gran Canaria,"1.254,62","510,15","413,23","331,24"
1,2019Q3,TOTAL,Gran Canaria,"1.239,57","520,76","365,67","353,14"
2,2019Q2,TOTAL,Gran Canaria,"1.124,73","474,48","340,77","309,48"
3,2019Q1,TOTAL,Gran Canaria,"1.059,74","478,40","259,42","321,92"
4,2018Q4,TOTAL,Gran Canaria,"1.205,52","528,63","341,18","335,71"


In [25]:
gasto_medio_islas.shape

(1680, 7)

## 2. ESTANDARIZAMOS ISLAS (Problema dificíl de ver)
**IMPORTANTE LEER:**

Nos hemos percatado en posteriores códigos para hacer merge (Horas y horas pensado que el merge no funcionaba como debería) que la columna isla tiene unos espaciados que imposibilitan La Unión posterior de datos de tipo isla así que vamos a tratarlo. esto nos enseña una vez más cómo el proceso de análisis de datos y tratamiento es un proceso en constante revisión y mejora que implica dar pasos hacia atrás debido a percatarnos de cosas que habíamos pasado previamente por alto.

In [26]:
gasto_medio_islas.isla.unique()

array(['   Gran Canaria', '   Fuerteventura', '   La Palma',
       '   Lanzarote', '   Tenerife', '   CANARIAS'], dtype=object)

In [27]:
replace_value(gasto_medio_islas, "isla", r'\s+Gran Canaria', r'Gran Canaria')
replace_value(gasto_medio_islas, "isla", r'\s+Fuerteventura', r'Fuerteventura')
replace_value(gasto_medio_islas, "isla", r'\s+La Palma', r'La Palma')
replace_value(gasto_medio_islas, "isla", r'\s+Lanzarote', r'Lanzarote')
replace_value(gasto_medio_islas, "isla", r'\s+Tenerife', r'Tenerife')
replace_value(gasto_medio_islas, "isla", r'\s+CANARIAS', r'CANARIAS')

Se ha cambiado el patron (\s+Gran Canaria) por el patron (Gran Canaria) en la columna isla
Se ha cambiado el patron (\s+Fuerteventura) por el patron (Fuerteventura) en la columna isla
Se ha cambiado el patron (\s+La Palma) por el patron (La Palma) en la columna isla
Se ha cambiado el patron (\s+Lanzarote) por el patron (Lanzarote) en la columna isla
Se ha cambiado el patron (\s+Tenerife) por el patron (Tenerife) en la columna isla
Se ha cambiado el patron (\s+CANARIAS) por el patron (CANARIAS) en la columna isla


Se han quitado los espacios...

In [28]:
gasto_medio_islas.isla.unique()

array(['Gran Canaria', 'Fuerteventura', 'La Palma', 'Lanzarote',
       'Tenerife', 'CANARIAS'], dtype=object)

## 3. ESTANDARIZAMOS TIPOS

In [29]:
gasto_medio_islas.dtypes

fecha                                                    period[Q-DEC]
pais                                                            object
isla                                                            object
total gasto por turista                                         object
gasto por turista alojamiento                                   object
gasto por turista transporte nacional / internacional           object
gasto por turista  otros                                        object
dtype: object

Pandas tiene un dtype especial para variables categoricas llamado `category`. El especificar a pandas que un campo es una categoria en vez de un object (un string generalmente) amplia las funcionalidades que podemos usar.Además, las categorias en general ocupan menos espacio en memoria (si su cardinalidad no es muy elevada).

In [30]:
gasto_medio_islas.pais = gasto_medio_islas.pais.astype("category")
gasto_medio_islas.isla = gasto_medio_islas.isla.astype("category")

hemos detectado que en la columna valor hay elementos que no son numeros, y son ".", ".." o "...", vamos a sustituir esto por "0"

In [31]:
columns=["total gasto por turista",
         "gasto por turista alojamiento",
        "gasto por turista transporte nacional / internacional",
        "gasto por turista  otros"]

for column in columns:
    gasto_medio_islas[column] = gasto_medio_islas[column].astype(str)
    
    replace_value(gasto_medio_islas, column, r'\.+', r'')   #reemplazamos los puntos por nada porque si no el metodo to_numeric da error

    #gasto_medio_islas[column] = gasto_medio_islas[column].apply(lambda x: float(x.split()[0].replace(',', '.')))
    #gasto_medio_islas[column] = gasto_medio_islas[column].apply(lambda x: float(x.split()[0].replace('.', '')))

    replace_value(gasto_medio_islas, column, r'(\d+),(\d+)', r'\1.\2') #Ahora remplazamos las comas por puntos

    #replace_value(gasto_medio_islas, column, r'nan', r'0')
    #replace_value(gasto_medio_islas, column, r'(\d+).(\d+).(\d+)', r'\1\2.\3') 
    gasto_medio_islas[column] = gasto_medio_islas[column].fillna(int(0))
    
    #gasto_medio_islas[column].astype(float)
    
    gasto_medio_islas[column] = pd.to_numeric(gasto_medio_islas[column].astype(str), downcast="float")  #pasamos la columna a numeros

    print("Done! Format: {}".format(gasto_medio_islas[column].dtypes))

Se ha cambiado el patron (\.+) por el patron () en la columna total gasto por turista
Se ha cambiado el patron ((\d+),(\d+)) por el patron (\1.\2) en la columna total gasto por turista
Done! Format: float32
Se ha cambiado el patron (\.+) por el patron () en la columna gasto por turista alojamiento
Se ha cambiado el patron ((\d+),(\d+)) por el patron (\1.\2) en la columna gasto por turista alojamiento
Done! Format: float32
Se ha cambiado el patron (\.+) por el patron () en la columna gasto por turista transporte nacional / internacional
Se ha cambiado el patron ((\d+),(\d+)) por el patron (\1.\2) en la columna gasto por turista transporte nacional / internacional
Done! Format: float32
Se ha cambiado el patron (\.+) por el patron () en la columna gasto por turista  otros
Se ha cambiado el patron ((\d+),(\d+)) por el patron (\1.\2) en la columna gasto por turista  otros
Done! Format: float32


In [32]:
gasto_medio_islas.head()

,fecha,pais,isla,total gasto por turista,gasto por turista alojamiento,gasto por turista transporte nacional / internacional,gasto por turista otros
0,2019Q4,TOTAL,Gran Canaria,1254.619995,510.149994,413.230011,331.239990
1,2019Q3,TOTAL,Gran Canaria,1239.569946,520.760010,365.670013,353.140015
2,2019Q2,TOTAL,Gran Canaria,1124.729980,474.480011,340.769989,309.480011
3,2019Q1,TOTAL,Gran Canaria,1059.739990,478.399994,259.420013,321.920013
4,2018Q4,TOTAL,Gran Canaria,1205.520020,528.630005,341.179993,335.709991


In [33]:
gasto_medio_islas.dtypes

fecha                                                    period[Q-DEC]
pais                                                          category
isla                                                          category
total gasto por turista                                        float32
gasto por turista alojamiento                                  float32
gasto por turista transporte nacional / internacional          float32
gasto por turista  otros                                       float32
dtype: object

# GUARDAMOS LOS CAMBIOS

In [34]:
rutaDatosTratados = "../PredictorGastoTuristico/DatosGuardados/"
gasto_medio_islas.to_csv(rutaDatosTratados+"gasto_medio_islas.csv", index = False, encoding='utf-8')